In [22]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

import os

import time
import datetime

In [23]:
def today_matchups():
        opts = Options()
        opts.add_argument("--headless")

        url = f"https://www.espn.com/nba/scoreboard/_/date/{datetime.datetime.today().strftime('%Y%m%d')}"

        chrome_driver = "C:\\Users\\Peter\\Downloads\\chromedriver_win32\\chromedriver.exe"

        driver = webdriver.Chrome(options=opts, service=Service(chrome_driver))

        driver.set_page_load_timeout(5)
        time.sleep(1)
        driver.set_window_size(2100, 9000)

        driver.get(url)

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        date = datetime.datetime.today().strftime("%m%d%y")

        games = soup.find_all("div", class_='Scoreboard__Row flex w-100 Scoreboard__Row__Main')
        games_data = []
        try:
            for game in games:
                game_data = {}
                away_tm = \
                game.find_all("div", class_="ScoreCell__TeamName ScoreCell__TeamName--shortDisplayName truncate db")[0]
                home_tm = \
                game.find_all("div", class_="ScoreCell__TeamName ScoreCell__TeamName--shortDisplayName truncate db")[1]
                game_data['Date'] = date
                try:
                    game_data['Time'] = game.find("div", class_="ScoreCell__Time ScoreboardScoreCell__Time h9 clr-gray-03").text
                except:
                    try:
                        game_data['Time'] = game.find("div", class_="ScoreCell__Time ScoreboardScoreCell__Time h9 clr-gray-01").text
                    except: 
                        try:
                            game_data['Time'] = game.find("div", class_="ScoreCell__Time ScoreboardScoreCell__Time h9 clr-negative").text
                        except: pass

                game_data["Away"] = away_tm.text
                # game_data["away_tm_abbr"] = team_names_nocity_dict[away_tm.text]

                try:
                    game_data["Away Score"] = game.find_all("div", class_="ScoreCell__Score h4 clr-gray-01 fw-heavy tar ScoreCell_Score--scoreboard pl2")[0].text
                except:
                    game_data['Away Score'] = 0

                game_data["Home"] = home_tm.text
                # game_data["home_tm_abbr"] = team_names_nocity_dict[home_tm.text]

                try:
                    game_data["Home Score"] = game.find_all("div", class_="ScoreCell__Score h4 clr-gray-01 fw-heavy tar ScoreCell_Score--scoreboard pl2")[1].text
                except:
                    game_data['Home Score'] = 0

                games_data.append(game_data)
        except: pass
        df = pd.DataFrame(games_data)
        driver.quit()

        html_table = df.to_html(index=False)

        return df

In [24]:
display(today_matchups())

,Date,Time,Away,Away Score,Home,Home Score
0,020523,3:38 - 1st,Cavaliers,21,Pacers,18
1,020523,6:00 PM,76ers,0,Knicks,0
2,020523,6:00 PM,Raptors,0,Grizzlies,0
3,020523,7:00 PM,Nuggets,0,Timberwolves,0
4,020523,7:00 PM,Kings,0,Pelicans,0
5,020523,Final,Magic,119,Hornets,113
